# Résolution de TPTW (travel problem with time window) avec métahéuristique du recuit simulé

In [1]:
# configuration
T_init = 1000
lbd = 0.999
nb_iter = 1000
path_to_instance = "data/inst1"

In [2]:
def evaluation_model(liste_prop, alpha=10):
    """
    évalue le modèle en fonction de la liste des propositions
    la formule choisie est la somme des distances plus une pénalité binaire pour chaque consigne de temps non respectée
    :rq: on utilise les variables globales instance et graphe_matrix
    """
    global instance, graphe_matrix
    score = 0
    assert liste_prop[0] == "1", "La première ville n'est pas 1 : {}".format(liste_prop)
    time_axis = 0
    for i, prop in enumerate(liste_prop):
        if i == 0: continue
        # on commence par aller jusqu'à la ville i
        time_axis += graphe_matrix[int(liste_prop[i-1])][int(liste_prop[i])]

        # si on est arrivé trop tôt, on attend
        if time_axis < instance[prop]["wstart"]:
            time_axis = instance[prop]["wstart"]

        # on ajoute le cout au score
        score += graphe_matrix[int(liste_prop[i-1])][int(prop)]

        # on ajoute une pénalité si on arrive trop tard (consigne de temps non respectée)
        if time_axis > instance[prop]["wend"]:
            score += alpha
    
    return score

    

## préparation de l'instance pré-résolution

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import bernoulli
from scoreEtudiant import *
from annexe import *
import tqdm

In [10]:
# calcul de la matrice des distances du graphe
instance = load_instance(path_to_instance)
graphe_matrix = compute_dist_mat(instance)


## Récuit simulé
On réalise ici l'algorithme du recuit-simulé.

In [11]:
# initialisation de l'algorithme
modele = list(instance.keys())
modele = ["1"] + random.sample(modele[1:], len(modele)-1)
print(modele)

temperature = T_init

modele_kept = modele.copy()
score_kept = evaluation_model(modele_kept)
score_kept

['1', '7', '16', '12', '9', '14', '15', '10', '2', '5', '18', '21', '20', '11', '13', '17', '19', '3', '6', '8', '4']


626.0

In [13]:
for iteration in tqdm.tqdm(range(nb_iter)):
    # on commence par appliquer une transformation aléatoire
    modele = transformee_pick_2(modele)

    # on détermine le score du modèle ainsi modifié
    score = evaluation_model(modele)

    # on accepte ou non la modification (recuit simulé)
    if score < score_kept:
        modele_kept = modele.copy()
    else:
        # principe du recuit simulé
        proba = np.exp(-(score-score_kept)/temperature)
        if bernoulli.rvs(proba, size=1):    # variable de bernoulli
            modele_kept = modele.copy()
            score_kept = score

    # on met à jour les historiques
    # TODO

    # on met à jour la température puis ça repart
    temperature *= lbd
print("[INFO] Simulation terminée")

100%|██████████| 1000/1000 [00:00<00:00, 36415.84it/s]

[INFO] Simulation terminée
